In [1]:
import sys # for automation and parallelization: set manual to false when run by a launcher
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])

# START

In [2]:
import sys
sys.path.insert(0, r'../../../quetzal')
from quetzal.model import stepmodel
import pandas as pd

SQLalchemy is not installed. No support for SQL output.


In [3]:
training_folder = '../../'
plot_path = training_folder + 'outputs/plot/'

In [4]:
distribution = stepmodel.read_zip(training_folder + r'model/{scen}/distribution.zip'.format(scen=scenario))
sm = stepmodel.read_zip(training_folder + r'model/base/walk_model.zip'.format(scen=scenario))

read_hdf: 100%|████████████████████████████████████████████████████████████████████████| 27/27 [00:01<00:00, 25.81it/s]


In [5]:
sm.volumes = distribution.volumes

# rail submodel

In [6]:
rail = sm.copy()
rail.links = rail.links.loc[rail.links['route_type'].isin(['tram', 'subway', 'rail'])] 

# strategy finder

In [7]:
rail.links['trip_id'] = rail.links['trip_id'].apply(lambda i: i.replace(':', '_').replace(' ', '_'))
rail.links= rail.links.loc[rail.links['a'] != rail.links['b']]

In [8]:
rail.nodes.index.name='index'
rail.step_strategy_finder()

100%|████████████████████████████████████████████████████████████████████████████████| 110/110 [01:15<00:00,  1.46it/s]


# assignment

In [9]:
rail.nodes.index.name='index'
rail.step_strategy_assignment('volume_pt_captive')

KeyError: 'volume_pt_captive'

In [ ]:
import os
from graphviz import Source
os.environ["PATH"] += os.pathsep + r'C:\Users\qchasserieau\dev\graphviz-2.38\release\bin/'
from IPython.core import display
import numpy as np
import networkx as nx

rail.links['trip_id'] = rail.links['trip_id'].apply(lambda i: i.replace(':', '_').replace(' ', '_'))

In [ ]:
header = """
ratio = fill; 
node [style="filled,rounded" ,shape="record", fontname = "calibri", fontsize=24,];
edge[ fontname = "calibri", fontsize=24];
ranksep = "0.5";
rankdir="LR";
"""
origin, destination='zone_70',  'zone_105'
a = rail.get_aggregated_edges(origin, destination)
a['l'] = 'p=' + np.round(a['p'], 2).astype(str) #+ '\nh:' + a['h'].astype(str)
a.loc[a['p'] == 1, 'l'] = ''

odg = nx.DiGraph()
for e in a.to_dict(orient='records'):
    odg.add_edge(e['i'], e['j'], label=e['l'])
name='test'
dot_string = nx.nx_pydot.to_pydot(odg).to_string().replace('{', '{' + header)
src = Source(dot_string,format='png')
display.Image(filename=src.render(name))

In [ ]:
self = rail
edges = self.optimal_strategy_edges[['i', 'j', 'f', 'c']].copy()
edges = edges.loc[self.optimal_strategy_sets.loc[destination]]
edges['ix'] = edges.index

# removing the edges that are non relevant (p<1e-6)
f_total = edges.groupby('i')[['f']].sum()
edges = pd.merge(edges, f_total, left_on='i', right_index=True, suffixes=['', '_total'])
edges['p'] = np.round(edges['f'] / edges['f_total'], 6)
edges = edges.loc[edges['p'] > 0]

In [ ]:
#restriction to the origin
g = nx.DiGraph()
for e in edges.to_dict(orient='records'):
    g.add_edge(e['i'], e['j'])

paths = list(nx.all_simple_paths(g, source=origin, target=destination))
nodes = set.union(*[set(p) for p in paths])
ode = edges.loc[edges['i'].isin(nodes) & edges['j'].isin(nodes)]

In [ ]:
rail.links.groupby(['a', 'b', 'trip_id']).first()